# KOSPI (1995.05.02 ~) 기업 category 

## WEB : KRX

## Colnames:

### 1. 종목코드, 업종명
### 2. 상장일, 상장 폐지일[있으면]


## RESULT : 1325(기업 수)

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd

import time
import requests
import os
from io import BytesIO

In [2]:
KOSPI = pd.read_csv("C:/Users/shic/Desktop/DH/price/950502_210712_KOSPI.csv",
                    dtype={"종목코드":str},index_col = 0)

C:\Users\shic\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
KOSPI

,종목코드,종목명,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,일자
0,009840,갑을,13000.0,100.0,0.78,12900.0,13100.0,12900.0,18920.0,2.462130e+08,5.395000e+10,4150000.0,19950502
1,011290,갑을방적,12500.0,-100.0,-0.79,12900.0,12900.0,11900.0,5790.0,7.256900e+07,3.816725e+10,3053380.0,19950502
2,000900,강원산업,14300.0,200.0,1.42,14100.0,14700.0,13900.0,131760.0,1.884797e+09,1.261257e+11,8819978.0,19950502
3,000905,강원산업(1우),8650.0,0.0,0.00,8650.0,8950.0,8640.0,6330.0,5.515500e+07,1.555614e+10,1798398.0,19950502
4,006590,강원은행,7750.0,390.0,5.30,7500.0,7750.0,7490.0,156040.0,1.185628e+09,1.646100e+11,21240000.0,19950502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032955,069260,휴켐스,24250.0,1850.0,8.26,22500.0,24900.0,22500.0,2273022.0,5.484788e+10,9.913058e+11,40878588.0,20210712
6032956,000540,흥국화재,4395.0,55.0,1.27,4350.0,4405.0,4350.0,102299.0,4.480097e+08,2.823464e+11,64242645.0,20210712
6032957,000547,흥국화재2우B,29350.0,950.0,3.35,28350.0,29500.0,28350.0,2118.0,6.150045e+07,4.508160e+09,153600.0,20210712
6032958,000545,흥국화재우,10300.0,250.0,2.49,10050.0,10350.0,10050.0,4342.0,4.438600e+07,7.910400e+09,768000.0,20210712


In [8]:
KOSPI.loc[:,["종목코드"]].drop_duplicates()

,종목코드
0,009840
1,011290
2,000900
3,000905
4,006590
...,...
5998698,383220
5999224,380440
6002460,383800
6002461,38380K


In [9]:
KOSPI.loc[:,["종목코드","종목명"]].drop_duplicates()

,종목코드,종목명
0,009840,갑을
1,011290,갑을방적
2,000900,강원산업
3,000905,강원산업(1우)
4,006590,강원은행
...,...,...
6028886,097520,엠씨넥스
6031161,344820,KCC글라스
6031198,001120,LX인터내셔널
6031199,108670,LX하우시스


In [258]:
# 현재 KOSPI 종목
req_header = {"Accept": "text/plain, */*; q=0.01",
              "Accept-Encoding": "gzip, deflate",
              "Accept-Language": "ko-KR,ko;q=0.9",
              "Connection": "keep-alive",
              "Content-Length": "526",
              "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
              "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==",
              "Host": "data.krx.co.kr",
              "Origin": "http://data.krx.co.kr",
              "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
              "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
              "X-Requested-With": "XMLHttpRequest"}
form_data = {
    "mktTpCd": "1",
    "tboxisuSrtCd_finder_listisu0_15": "전체",
    "isuSrtCd": "ALL",
    "isuSrtCd2": "ALL",
    "codeNmisuSrtCd_finder_listisu0_15": "",
    "param1isuSrtCd_finder_listisu0_15": "",
    "sortType": "A",
    "stdIndCd": "ALL",
    "sectTpCd": "ALL",
    "parval": "ALL",
    "mktcap": "ALL",
    "acntclsMm": "ALL",
    "tboxmktpartcNo_finder_designadvser0_15": "",
    "mktpartcNo":"",
    "mktpartcNo2": "",
    "codeNmmktpartcNo_finder_designadvser0_15": "",
    "param1mktpartcNo_finder_designadvser0_15": "",
    "condListShrs": "1",
    "listshrs":  "",
    "listshrs2": "",
    "condCap": "1",
    "cap": "",
    "cap2": "",
    "share": "1",
    "money": "1",
    "csvxls_isNo": "false",
    "name": "fileDown",
    "url": "dbms/MDC/STAT/standard/MDCSTAT03402"
}

down_req_header = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                   "Accept-Encoding": "gzip, deflate",
                   "Accept-Language": "ko-KR,ko;q=0.9",
                   "Cache-Control": "max-age=0",
                   "Connection": "keep-alive",
                   "Content-Length": "1007",
                   "Content-Type": "application/x-www-form-urlencoded",
                   "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==",
                   "Host": "data.krx.co.kr",
                   "Origin": "http://data.krx.co.kr",
                   "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
                   "Upgrade-Insecure-Requests": "1",
                   "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [259]:
r = requests.post("http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd",form_data,req_header)
r = requests.post("http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd",
                  {"code" : r.text},
                  down_req_header)
cate = pd.read_csv(BytesIO(r.content),encoding="CP949",
                  dtype={"종목코드":str})

In [260]:
cate = cate.loc[:,["종목코드","종목명","업종명"]]

In [261]:
cate

,종목코드,종목명,업종명
0,095570,AJ네트웍스,산업용 기계 및 장비 임대업
1,006840,AK홀딩스,기타 금융업
2,027410,BGF,기타 금융업
3,282330,BGF리테일,종합 소매업
4,138930,BNK금융지주,기타 금융업
...,...,...,...
804,079980,휴비스,화학섬유 제조업
805,005010,휴스틸,1차 철강 제조업
806,069260,휴켐스,기타 화학제품 제조업
807,000540,흥국화재,보험업


In [262]:
req_header = {"Accept": "text/plain, */*; q=0.01",
              "Accept-Encoding": "gzip, deflate",
              "Accept-Language": "ko-KR,ko;q=0.9",
              "Connection": "keep-alive",
              "Content-Length": "97",
              "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
              "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==",
              "Host": "data.krx.co.kr",
              "Origin": "http://data.krx.co.kr",
              "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
              "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
              "X-Requested-With": "XMLHttpRequest"}
form_data = {
    "mktId": "STK",
    "share": "1",
    "csvxls_isNo": "false",
    "name": "fileDown",
    "url": "dbms/MDC/STAT/standard/MDCSTAT01901"
}

down_req_header = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                   "Accept-Encoding": "gzip, deflate",
                   "Accept-Language": "ko-KR,ko;q=0.9",
                   "Cache-Control": "max-age=0",
                   "Connection": "keep-alive",
                   "Content-Length": "237",
                   "Content-Type": "application/x-www-form-urlencoded",
                   "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==",
                   "Host": "data.krx.co.kr",
                   "Origin": "http://data.krx.co.kr",
                   "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
                   "Upgrade-Insecure-Requests": "1",
                   "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [263]:
r = requests.post("http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd",form_data,req_header)
r = requests.post("http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd",
                  {"code" : r.text},
                  down_req_header)
cate_date = pd.read_csv(BytesIO(r.content),encoding="CP949",
                  dtype={"종목코드":str})
cate_date

,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수
0,KR7095570008,095570,AJ네트웍스보통주,AJ네트웍스,"AJ Networks Co.,Ltd.",2015/08/21,KOSPI,주권,NaN,보통주,1000,46822295
1,KR7006840003,006840,AK홀딩스보통주,AK홀딩스,"AK Holdings, Inc.",1999/08/11,KOSPI,주권,NaN,보통주,5000,13247561
2,KR7282330000,282330,BGF리테일보통주,BGF리테일,BGF Retail,2017/12/08,KOSPI,주권,NaN,보통주,1000,17283906
3,KR7027410000,027410,BGF보통주,BGF,BGF,2014/05/19,KOSPI,주권,NaN,보통주,1000,95716791
4,KR7138930003,138930,BNK금융지주보통주,BNK금융지주,BNK Financial Group Inc.,2011/03/30,KOSPI,주권,NaN,보통주,5000,325935246
...,...,...,...,...,...,...,...,...,...,...,...,...
923,KR7069260008,069260,휴켐스보통주,휴켐스,HuchemsFineChemicalCorporation,2002/10/07,KOSPI,주권,NaN,보통주,1000,40878588
924,KR7000541003,000545,흥국화재해상보험1우선주,흥국화재우,HeungkukFire&MarineIns(1P),1990/03/20,KOSPI,주권,NaN,구형우선주,5000,768000
925,KR7000542001,000547,흥국화재해상보험2우선주(신형),흥국화재2우B,HeungkukFire&MarineIns(2PB),1999/08/09,KOSPI,주권,NaN,신형우선주,5000,153600
926,KR7000540005,000540,흥국화재해상보험보통주,흥국화재,HeungkukFire&MarineInsurance,1974/12/05,KOSPI,주권,NaN,보통주,5000,64242645


In [272]:
cate = pd.merge(cate,
         cate_date.loc[:,["단축코드","상장일"]],
         left_on="종목코드",
         right_on="단축코드",
        how = "left")

In [273]:
cate

,종목코드,종목명,업종명,단축코드,상장일
0,095570,AJ네트웍스,산업용 기계 및 장비 임대업,095570,2015/08/21
1,006840,AK홀딩스,기타 금융업,006840,1999/08/11
2,027410,BGF,기타 금융업,027410,2014/05/19
3,282330,BGF리테일,종합 소매업,282330,2017/12/08
4,138930,BNK금융지주,기타 금융업,138930,2011/03/30
...,...,...,...,...,...
804,079980,휴비스,화학섬유 제조업,079980,2012/02/23
805,005010,휴스틸,1차 철강 제조업,005010,1973/06/29
806,069260,휴켐스,기타 화학제품 제조업,069260,2002/10/07
807,000540,흥국화재,보험업,000540,1974/12/05


In [228]:
# 상장 폐지된 종목
def krx_cate(STDATE,ENDDATE):
    gen_req_headers = {"Accept": "text/plain, */*; q=0.01",
                         "Accept-Encoding": "gzip, deflate",
                         "Accept-Language": "ko-KR,ko;q=0.9",
                         "Connection": "keep-alive",
                         "Content-Length": "263",
                         "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                         "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==; finder_listdelisu_finderCd=finder_listdelisu; finder_listdelisu_param1=STK; finder_listdelisu_tbox=011290%2F%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeNm=%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeVal=KR7011290004; finder_comnm_finderCd=finder_comnm; finder_comnm_tbox=005930%2F%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeNm=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeVal=00593; finder_comnm_codeVal2=KR7005930003; finder_listdelisu_codeVal2=KR7011290004",
                         "Host": "data.krx.co.kr",
                         "Origin": "http://data.krx.co.kr",
                         "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
                         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                         "X-Requested-With": "XMLHttpRequest"}

    gen_form_data = {
                    "mktId": "STK",
                    "tboxisuCd_finder_listdelisu0_1": "전체",
                    "isuCd": "ALL",
                    "isuCd2": "ALL",
                    "codeNmisuCd_finder_listdelisu0_1": "" ,
                    "param1isuCd_finder_listdelisu0_1": "",
                    "strtDd": STDATE,
                    "endDd": ENDDATE,
                    "share": "1",
                    "csvxls_isNo": "true",
                    "name": "fileDown",
                    "url": "dbms/MDC/STAT/issue/MDCSTAT23801"
    }

    down_req_headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "ko-KR,ko;q=0.9",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Content-Length": "567",
        "Content-Type": "application/x-www-form-urlencoded",
        "Cookie": "__utma=139639017.1771199946.1626216000.1626216000.1626216000.1; __utmc=139639017; __utmz=139639017.1626216000.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __smVisitorID=mqaSX0t6UrL; JSESSIONID=NzCZgfqV3jJHxF7ctkxsPFQUbRUzyVwmwczqOaMSqZMWCESSJwIgU1oaGx4huXec.bWRjX2RvbWFpbi9tZGNvd2FwMS1tZGNhcHAxMQ==; finder_listdelisu_finderCd=finder_listdelisu; finder_listdelisu_param1=STK; finder_listdelisu_tbox=011290%2F%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeNm=%EA%B0%91%EC%9D%84%EB%B0%A9%EC%A0%81; finder_listdelisu_codeVal=KR7011290004; finder_comnm_finderCd=finder_comnm; finder_comnm_tbox=005930%2F%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeNm=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90; finder_comnm_codeVal=00593; finder_comnm_codeVal2=KR7005930003; finder_listdelisu_codeVal2=KR7011290004",
        "Host": "data.krx.co.kr",
        "Origin": "http://data.krx.co.kr",
        "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    r = requests.post("http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd",gen_form_data,gen_req_headers)
    r = requests.post("http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd",
                  {"code" : r.text},
                  down_req_headers)
    return pd.read_csv(BytesIO(r.content),encoding="CP949",
                      dtype={"종목코드":str,
                            "이전상장후_종목코드":str})

In [229]:
TMP = pd.DataFrame(columns=['번호', '종목코드', '종목명', '시장구분', '증권구분', '주식종류', '상장일', '폐지일', '폐지사유','정리매매기간_시작일', '정리매매기간_종료일', '업종', '액면가', '상장주식수', '이전상장후_종목코드','이전상장후_종목명'])

In [230]:
for yyyy in range(1995,2022):
    std = str(yyyy) + "0502"
    end = str(yyyy+1) + "0501"
    TMP = TMP.append(krx_cate(std,end),
              ignore_index=True)
TMP = TMP.rename(columns={'업종': '업종명'})

In [243]:
TMP = TMP.loc[:,["종목코드","업종명","상장일","폐지일","폐지사유"]]
TMP = TMP[TMP.loc[:,"업종명"] == TMP.loc[:,"업종명"]]
TMP.index = list(range(TMP.shape[0]))

In [274]:
TMP

,종목코드,업종명,상장일,폐지일,폐지사유
160,003497,운수창고업,1998/04/25,2001/04/16,NaN
166,014080,전기전자,1994/01/31,2001/04/26,공시서류 미제출(분기보고서) 후 차기보고서 미제출
167,017320,전기전자,1989/06/05,2001/04/26,공시서류 미제출(반기보고서) 후 차기보고서 미제출
168,017325,전기전자,1989/12/23,2001/04/26,공시서류 미제출(반기보고서) 후 차기보고서 미제출
170,001400,화학,1976/06/08,2001/04/24,회사정리절차 폐지 결정
...,...,...,...,...,...
1623,099350,기타금융,2008/02/01,2020/07/01,존속기간 만료
1624,005450,건설업,1978/07/28,2020/06/30,감사의견 의견거절 2년 계속
1625,083370,서비스업,2005/08/29,2020/06/23,존속기간 만료
1629,103130,전기전자,2010/06/30,2020/06/01,감사의견 의견거절 2년 계속


In [308]:
cate["폐지일"] = "Not Delisting"
cate["폐지사유"] = "Not Delisting"
cate = cate.loc[:,["종목코드","업종명","상장일","폐지일","폐지사유"]]

In [295]:
TMP["폐지사유"] = pd.Series([x if x == x else "No searched" for x in TMP.폐지사유])

In [311]:
TMP

,종목코드,업종명,상장일,폐지일,폐지사유
0,003497,운수창고업,1998/04/25,2001/04/16,No searched
1,014080,전기전자,1994/01/31,2001/04/26,공시서류 미제출(분기보고서) 후 차기보고서 미제출
2,017320,전기전자,1989/06/05,2001/04/26,공시서류 미제출(반기보고서) 후 차기보고서 미제출
3,017325,전기전자,1989/12/23,2001/04/26,공시서류 미제출(반기보고서) 후 차기보고서 미제출
4,001400,화학,1976/06/08,2001/04/24,회사정리절차 폐지 결정
...,...,...,...,...,...
511,099350,기타금융,2008/02/01,2020/07/01,존속기간 만료
512,005450,건설업,1978/07/28,2020/06/30,감사의견 의견거절 2년 계속
513,083370,서비스업,2005/08/29,2020/06/23,존속기간 만료
514,103130,전기전자,2010/06/30,2020/06/01,감사의견 의견거절 2년 계속


In [315]:
cate = cate.append(TMP,
                   ignore_index=True)

In [316]:
cate

,종목코드,업종명,상장일,폐지일,폐지사유
0,095570,산업용 기계 및 장비 임대업,2015/08/21,Not Delisting,Not Delisting
1,006840,기타 금융업,1999/08/11,Not Delisting,Not Delisting
2,027410,기타 금융업,2014/05/19,Not Delisting,Not Delisting
3,282330,종합 소매업,2017/12/08,Not Delisting,Not Delisting
4,138930,기타 금융업,2011/03/30,Not Delisting,Not Delisting
...,...,...,...,...,...
1320,099350,기타금융,2008/02/01,2020/07/01,존속기간 만료
1321,005450,건설업,1978/07/28,2020/06/30,감사의견 의견거절 2년 계속
1322,083370,서비스업,2005/08/29,2020/06/23,존속기간 만료
1323,103130,전기전자,2010/06/30,2020/06/01,감사의견 의견거절 2년 계속


In [320]:
#cate.to_csv("C:/Users/shic/Desktop/DH/price/Corp_category.csv")

In [322]:
pd.read_csv("C:/Users/shic/Desktop/DH/price/Corp_category.csv",
           dtype={"종목코드":str},index_col = 0)

,종목코드,업종명,상장일,폐지일,폐지사유
0,095570,산업용 기계 및 장비 임대업,2015/08/21,Not Delisting,Not Delisting
1,006840,기타 금융업,1999/08/11,Not Delisting,Not Delisting
2,027410,기타 금융업,2014/05/19,Not Delisting,Not Delisting
3,282330,종합 소매업,2017/12/08,Not Delisting,Not Delisting
4,138930,기타 금융업,2011/03/30,Not Delisting,Not Delisting
...,...,...,...,...,...
1320,099350,기타금융,2008/02/01,2020/07/01,존속기간 만료
1321,005450,건설업,1978/07/28,2020/06/30,감사의견 의견거절 2년 계속
1322,083370,서비스업,2005/08/29,2020/06/23,존속기간 만료
1323,103130,전기전자,2010/06/30,2020/06/01,감사의견 의견거절 2년 계속
